In [8]:
import pandas as pd
ratings_movies=pd.read_csv('data/ratings_movies.csv', sep=',')
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

# Задание 1.
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма. 
# У скольких фильмов не указан год их выпуска?
ratings_movies['year_release']=ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

# Задание 2. Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей? 
# В качестве ответа запишите название этого фильма без указания года его выпуска.
mask1=ratings_movies['year_release']==1999 #обозначил маску на 1999
ratings_movies[mask1].groupby('title')['rating'].mean().sort_values()

# Задание 3.
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
mask2=ratings_movies['year_release']==2010
ratings_movies[mask2].groupby('genres')['rating'].mean().sort_values()

# Задание 4.
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) 
# фильмов? В качестве ответа запишите идентификатор этого пользователя.
ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

# Задание 5.
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
# можно воспользоваться методом agg() на сгруппированных данных.
ratings_movies.groupby('userId')['rating'].agg( #группировка по id в зависимости от рейтинга
    ['count', 'mean'] # вбиваем кол-во значений и средний 
).sort_values(['count', 'mean'], ascending=[True, False]) # сорт по кол-ву и среднему, сорт для значений указываем по от большего к меньшему, сорт по среднему наоборот

# Задание 6.
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных 
# ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов)
mask3=ratings_movies['year_release']==2018 # ввел маску
group1=ratings_movies[mask3].groupby('genres')['rating'].agg( # изначально взял по 2018 и сгрупп по жанру от рэйтинга, в кач арг среднее и кол-во знач
    ['mean','count']
)
group1[group1['count']>10].sort_values( # ???? вроде как по пред групировке но значений больше 10 и далее сорт по среднему в порядке возрастания
    by='mean',
    ascending=False
)

# Задание 7.
# Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма 
# от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:
ratings_movies['date']=pd.to_datetime(ratings_movies['date']) # преобразовал в формат год признак date
ratings_movies['year_rating']=ratings_movies['date'].dt.year # добавил новый признак по году
pivot=ratings_movies.pivot_table( # добавил сводник
    index='year_rating', # по году
    columns='genres', # шапка по жанрам
    values='rating', # значения по рейту
    aggfunc='mean' # функция по среднему
)
display(pivot)

# Задание 8.
# !!!! Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах, но не оказалось информации о продуктах, на 
# которых заказов ещё не поступало. 
# Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?
orders=pd.read_csv('data/orders.csv', sep=';')
products=pd.read_csv('data/products.csv', sep=';')
order_product=orders.merge(
    products,
    left_on='ID товара',
    right_on='Product_ID',
    how='left')
order_product[order_product['Name'].isna()]

# Задание 9.
# На какой товар была произведена отмена? В качестве ответа запишите название этого товара (Name).
order_product[order_product['Отменен']=='Да']["Name"]

# Задание 10.
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# В ответ запишите идентификатор этого покупателя (ID Покупателя).
# Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.
order_product['csb']=order_product['Price']*order_product['Количество']
order_product[order_product['Оплачен']=='Да'].groupby('ID Покупателя')['csb'].sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: csb, dtype: float64